In [1]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict
import requests
from bs4 import BeautifulSoup
import time

In [2]:
def vancouver_grid(head, divisions=15):
    van_geo_info_url = 'https://opendata.vancouver.ca/api/explore/v2.1/catalog/datasets/city-boundary/records?limit=20'
    response = requests.get(van_geo_info_url,headers=head)
    geo_data = response.json()
    contour = geo_data['results'][0]['geom']['geometry']['coordinates']

    longitudes = [coord[0] for sublist in contour for coord in sublist]
    latitudes = [coord[1] for sublist in contour for coord in sublist]

    max_longitude = max(longitudes)
    min_longitude = min(longitudes)
    max_latitude = max(latitudes)
    min_latitude = min(latitudes)

    longitude_step = (max_longitude - min_longitude) / divisions
    latitude_step = (max_latitude - min_latitude) / divisions

    coord_boxes = []
    for i in range(divisions):
        for j in range(divisions):
            box_min_lat = round(min_latitude + j * latitude_step, 5)
            box_max_lat = round(min_latitude + (j + 1) * latitude_step, 5)
            box_min_lon = round(min_longitude + i * longitude_step, 5)
            box_max_lon = round(min_longitude + (i + 1) * longitude_step, 5)

            box_str = f"{box_min_lat}:{box_max_lat}:{box_min_lon}:{box_max_lon}"
            coord_boxes.append(box_str)
    
    return  coord_boxes

In [3]:
def listing_count(head, coord_box):

    viewport_url = f"https://www.redfin.ca/bc/vancouver/filter/viewport={coord_box}"
    resp = requests.get(viewport_url, headers=head)

    if resp.status_code != 200:
        raise Exception("Failing in webpage requests")
    
    soup = BeautifulSoup(resp.text, 'html.parser')
    if soup.find('div', {'class': 'HomeViews reversePosition'}).find('h2'):
        return 'no_listing'
    
    listing_summary = soup.find('div', {'class':"homes summary reversePosition"})
    select_listing_count, total_listing_count = re.findall(r'\d{1,10}(?:,\d{1,10})*', listing_summary.text)
    select_listing_count, total_listing_count = int(select_listing_count), int(total_listing_count.replace(',',''))
    
    return viewport_url, select_listing_count, total_listing_count 

In [4]:
def crawling_redfin(head, viewport_url, page):
    
    target_url = viewport_url + "/page-" + str(page)
    resp = requests.get(target_url,headers=head)
    
    if resp.status_code != 200:
        raise Exception("Failing in webpage requests")
    
    soup=BeautifulSoup(resp.text,'html.parser')
    soup_boxes = soup.find_all("div",{"class":"HomeCardContainer"})

    return  soup_boxes

In [5]:
def key_metric_extraction(soup_boxes, real_estate_info):
    
    incomplete_idx = []

    for i, box in enumerate(soup_boxes):
        try:
            address = box.find('address').text[:(-23)]
            real_estate_info['address'].append(address)
        except: 
            real_estate_info['address'].append(np.nan)
            incomplete_idx.append(i)
        try:
            zip_code = box.find('address').text[-7:]
            real_estate_info['zip_code'].append(zip_code)
        except: 
            real_estate_info['zip_code'].append(np.nan)
            incomplete_idx.append(i)        
        try:
            price = box.find('span', {'class':'bp-Homecard__Price--value'}).text
            real_estate_info['price'].append(price)
        except: 
            real_estate_info['price'].append(np.nan)
            incomplete_idx.append(i)
        try:
            bed = box.find('span', {'class':'bp-Homecard__Stats--beds text-nowrap'}).text
            real_estate_info['bed'].append(bed)
        except: 
            real_estate_info['bed'].append(np.nan)
            incomplete_idx.append(i)   
        try:
            bath = box.find('span', {'class':'bp-Homecard__Stats--baths text-nowrap'}).text
            real_estate_info['bath'].append(bath)
        except: 
            real_estate_info['bath'].append(np.nan)
            incomplete_idx.append(i)   
        try:
            sqr_footage = box.find('span', {'class':'bp-Homecard__LockedStat--value'}).text
            real_estate_info['sqr_footage'].append(sqr_footage)
        except:
            real_estate_info['sqr_footage'].append(np.nan)
            incomplete_idx.append(i)   
        try:
            property_link = "https://www.redfin.com"+box.find("a").get('href')
            real_estate_info['property_link'].append(property_link)
        except:
            real_estate_info['property_link'].append(np.nan)
            incomplete_idx.append(i)

    return incomplete_idx


In [6]:
def calculate_min_pages(total_count, items_per_page):
    return (total_count + items_per_page - 1) // items_per_page

In [50]:
def extracting_by_batch(head, batch_num, test_batch_index=[0,1]):
    big_coord_boxes = []
    real_estate_info = defaultdict(list)
    missing_entries = defaultdict(list)

    coord_boxes = vancouver_grid(head)
    coord_box_batch = np.array_split(coord_boxes, batch_num)
    for i in range(test_batch_index[0], test_batch_index[1]):
        batch = coord_box_batch[i]
        for coord_box in batch:
            listing_info = listing_count(head, coord_box)
            time.sleep(1)
            if listing_info == 'no_listing':
                print(f"Batch {i} has no listings.")
                continue
            else:
                viewport_url, select_listing_count, total_listing_count = listing_info
                if select_listing_count != total_listing_count:
                    big_coord_boxes.append(coord_boxes)
                    continue
                else:
                    max_page = calculate_min_pages(select_listing_count, items_per_page=9)
                    missing = defaultdict(list)
                    for page in range(1, max_page):
                        soup_boxes = crawling_redfin(head, viewport_url, page)
                        incomplete_idx = key_metric_extraction(soup_boxes, real_estate_info)
                        if incomplete_idx:
                            missing[f'page_{page}'].append(incomplete_idx)
                        time.sleep(1)
                    missing_entries[coord_box].append(missing)
    return real_estate_info, missing_entries, big_coord_boxes

In [51]:
head = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
real_estate_info, missing_entries, big_coord_boxes = extracting_by_batch(head, batch_num=40, test_batch_index=[20,21])

Batch 20 has no listings.


In [54]:
pd.DataFrame(real_estate_info)

,address,zip_code,price,bed,bath,sqr_footage,property_link
0,455 SW Marine Dr #615,V5X 0H3,"$799,000",2 beds,2 baths,880,https://www.redfin.com/bc/vancouver/455-SW-Mar...
1,8131 Nunavut Lane #1004,V5X 0E2,"$765,000",2 beds,2 baths,765,https://www.redfin.com/bc/vancouver/8131-Nunav...
2,8131 Nunavut Lane #503,V5X 0E2,"$549,000",1 bed,1 bath,446,https://www.redfin.com/bc/vancouver/8131-Nunav...
3,455 SW Marine Dr #505,V5X 0H3,"$883,888",2 beds,2 baths,753,https://www.redfin.com/bc/vancouver/455-SW-Mar...
4,489 Interurban Way #1608,V5X 0C7,"$485,000",1 bed,1 bath,483,https://www.redfin.com/bc/vancouver/489-Interu...
...,...,...,...,...,...,...,...
743,5168 Cambie St #606,V5Z 0K7,"$1,079,900",1 bed,1 bath,582,https://www.redfin.com/bc/vancouver/5168-Cambi...
744,5168 Cambie St #602,V5Z 0K7,"$1,929,900",2 beds,2 baths,985,https://www.redfin.com/bc/vancouver/5168-Cambi...
745,5168 Cambie St #607,Z5Z 0K7,"$1,149,900",1 bed,1 bath,595,https://www.redfin.com/bc/vancouver/5168-Cambi...
746,508 W 29th Ave #206,V5Z 2H7,"$739,000",1 bed,1 bath,593,https://www.redfin.com/bc/vancouver/508-W-29th...


In [ ]:
head = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
coord_boxes = vancouver_grid(head)
real_estate_info = defaultdict(list)
big_coord_boxes = []
missing_entries = defaultdict(list)

for i in range(100,101):
    coord_box = coord_boxes[i]
    listing_info = listing_count(head, coord_box)
    time.sleep(1)
    if listing_info == 'no_listing':
        continue
    else:
        viewport_url, select_listing_count, total_listing_count = listing_info
        if select_listing_count != total_listing_count:
            big_coord_boxes.append(coord_box)
            continue
        else:
            max_page = calculate_min_pages(select_listing_count, items_per_page=9)
            missing = defaultdict(list)
            for page in range(1, max_page):
                soup_boxes = crawling_redfin(head, viewport_url, page)
                incomplete_idx = key_metric_extraction(soup_boxes, real_estate_info)
                missing[f'page_{page}'].append(incomplete_idx)
                time.sleep(1)
            missing_entries[f'coord_box_{i}'].append(missing)

In [8]:
collected_data = pd.DataFrame(real_estate_info)
collected_data.to_csv('../data/ancouver_real_estate.csv', index=False)